<a href="https://colab.research.google.com/github/SongyangZhou/TensorFlow_Implementing/blob/main/02-Main%20Algorithms%20Implementing%20In%20TensorFLow%20-%20DNN%20Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifier in TensorFlow
- Environment preparation

In [2]:
%tensorflow_version 2.x

# make sure it is compatible with both Python 2 and Python 3
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt



Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


- Load the data set and do some data preprocessing

In [3]:
# load dataset
# the way to load a link file
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

df_train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)# training dataset
df_eval = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0) # test dataset

# split predictors and label.
x_train = df_train.drop(columns=['Species'])
y_train = df_train['Species']
x_eval = df_eval.drop(columns=['Species'])
y_eval = df_eval['Species']

print(x_train.head()) # Check the x_train. The lable "survived" is excluded.
print(y_train.head()) # Check the y_train
print(x_eval.head()) # Check the x_eval
print(y_eval.head()) # Check the y_eval
print(x_train.shape)
print(y_train.shape)
print(x_eval.shape)
print(y_eval.shape)

2194/2194 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
573/573 ━━━━━━━━━━━━━━━━━━━━ 0s 6us/step
   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3
0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64
   SepalLength  SepalWidth  PetalLength  PetalWidth
0          5.9         3.0          4.2         1.5
1          6.9         3.1          5.4         2.1
2          5.1         3.3          1.7         0.5
3          6.0         3.4          4.5         1.6
4          5.5         2.5          4.0         1.3
0    1
1    2
2    0
3    1
4    1
Name: Species, dtype: int64
(120, 4)
(120,)
(30, 4)
(30,)


In [4]:
# for classification, the Keras require the labels to be converted into one-hot encoding.
y_train_onehot = to_categorical(y_train, num_classes=3)
y_eval_onehot = to_categorical(y_eval, num_classes=3)
y_train_onehot[0:5,] # check the onehot ecoding result.

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]])

- Bulid and compile the model
the num of neurons in each layer are somewhat heristic and based on experimentation. In some specific numbers may vary, now we don't need to care about that. They are hyperparameters we need to optimize in specific case.

In [5]:
# build the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(x_train.shape[1],)), # define the first hidden layer and the input data shape of this layer.
    Dense(64, activation='relu'), # 2nd hidden layer
    Dense(32, activation='relu'), # 3rd hidden layer
    Dense(3, activation='softmax')
])

# compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


- train the model

In [6]:
history = model.fit(x_train, y_train_onehot, epochs=100, batch_size=16, validation_data=(x_eval, y_eval_onehot))
# in this case we use test dataset to validate the model we trained, it doesn't we use test dataset to train the model.
# in practice, we usually devide our dataset into training set, dev set and test set. the dev set can be used for validation.


Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.3108 - loss: 1.2369 - val_accuracy: 0.2667 - val_loss: 1.1025
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4430 - loss: 0.9770 - val_accuracy: 0.5333 - val_loss: 0.9240
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6815 - loss: 0.8378 - val_accuracy: 0.6667 - val_loss: 0.7882
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8141 - loss: 0.7012 - val_accuracy: 0.5667 - val_loss: 0.7180
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7087 - loss: 0.6018 - val_accuracy: 0.7667 - val_loss: 0.6193
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8706 - loss: 0.5522 - val_accuracy: 0.8000 - val_loss: 0.5543
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8102 - loss: 0.4606 - val_accuracy: 0.7000 - val_loss: 0.5184
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9331 - loss: 0.3972 - val_accuracy: 1.0000 - val_loss: 

- evaluate the model with test dataset

In [11]:
# evlauate the model with loss and accuarcy on the test dataset
test_loss, test_acc = model.evaluate(x_eval, y_eval_onehot)
print('Test accuracy:', test_acc)

# predict. you can change another dataset instead of test dataset
# the result is returned as the possibility of each class
y_pred = model.predict(x_eval)
print(y_pred[0:5,])
# covert the possibility into exact class
y_pred_class = np.argmax(y_pred, axis=1)
print(y_pred_class[0:5])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9667 - loss: 0.0804
Test accuracy: 0.9666666388511658
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[[1.3565519e-04 9.8905522e-01 1.0809111e-02]
 [1.2440381e-07 2.1639980e-02 9.7835988e-01]
 [9.9958354e-01 4.1640672e-04 6.9647560e-10]
 [7.8611731e-05 9.7769541e-01 2.2225985e-02]
 [2.0746895e-04 9.6679419e-01 3.2998316e-02]]
[1 2 0 1 1]


Now, we're all done for the common and basic classiciation processing in TensorFlow. Hope you enjoy it and leave a message if you like it.